In [ ]:
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# загрузка csv-файла
df = pd.read_csv('unique.csv')

# предобработка данных
def preprocess_text(text):
    df['title'] = df['title'].apply(lambda x: x.lower())
    df['title'] = df['title'].apply(lambda x: x.replace(r'\W+', ' ').replace(r'\d+', ''))

    return text

df['title'] = df['title'].apply(preprocess_text)

# обучение модели doc2vec
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(df['title'])]
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

# генерация эмбеддингов
embeddings = []
for i, doc in enumerate(df['title']):
    embeddings.append(model.infer_vector(doc.split()))

# функция генерации рекомендаций по заголовку
def get_recommendations(sample_header):
    # предобработка заголовка
    sample_header = preprocess_text(sample_header)
    sample_header_embedding = model.infer_vector(sample_header.split())

    # вычисление косинусное сходство с другими заголовками
    similarities = []
    for i, header in enumerate(df['title']):
        header_embedding = embeddings[i]
        similarity = cosine_similarity([sample_header_embedding], [header_embedding])[0][0]
        similarities.append((header, similarity))

    # сортировка рекомендаций в нисходящем порядке
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # возврат 5 рекомендаций
    return [(header, similarity) for header, similarity in sorted_similarities[:5]]

In [ ]:
t1 = 'Макрон объявил о роспуске Нацсобрания и назначил досрочные выборы'

t2 = 'Макрон назвал дату открытия отреставрированного Нотр-Дама'

t3 = 'Сестра Ким Чен Ына предупредила Сеул о риске эскалации «мусорной войны»'

t4 = 'В МИД России вызывали посла Австрии для выяснения претензий Вены'

t5 = 'Гладков предупредил жителей Белгородской области о «сложной опасности»'

In [ ]:
recommendations = get_recommendations(t1)

print(f"Рекомендации по запросу '{t1}':")
for i, (headline, score) in enumerate(recommendations, start=1):
    print(f"{i}. {headline} (score: {score})")

Рекомендации по запросу 'Макрон объявил о роспуске Нацсобрания и назначил досрочные выборы':
1. bloomberg назвал целью саммита в швейцарии вовлечение россии в переговоры  (score: 0.21420714259147644)
2. мид сообщил, что россии не нравится отношение талибов к женщинам  (score: 0.18457981944084167)
3. nbc узнал о 8 задержанных в сша за связи с иг граждан таджикистана  (score: 0.18085351586341858)
4. адвокат назвал условия возращения иванову арестованного имущества  (score: 0.1731259673833847)
5. делимханов пообещал даванкову объяснить дресс-код женщин в россии  (score: 0.16852284967899323)


In [ ]:
recommendations = get_recommendations(t2)

print(f"Рекомендации по запросу '{t2}':")
for i, (headline, score) in enumerate(recommendations, start=1):
    print(f"{i}. {headline} (score: {score})")

Рекомендации по запросу 'Макрон назвал дату открытия отреставрированного Нотр-Дама':
1. белый дом заявил о нежелании байдена провоцировать третью мировую войну  (score: 0.2916637659072876)
2. полиция применила светошумовые гранаты против протестующих в армении  (score: 0.21431460976600647)
3. президентом исландии стала бывший директор компании ричарда брэнсона  (score: 0.21216318011283875)
4. трамп пообещал рассекретить документы об убийстве кеннеди  (score: 0.20463286340236664)
5. гладков рассказал о разрушениях после атаки на окрестности белгорода  (score: 0.19379360973834991)


In [ ]:
recommendations = get_recommendations(t3)

print(f"Рекомендации по запросу '{t3}':")
for i, (headline, score) in enumerate(recommendations, start=1):
    print(f"{i}. {headline} (score: {score})")

Рекомендации по запросу 'Сестра Ким Чен Ына предупредила Сеул о риске эскалации «мусорной войны»':
1. в мид россии вызывали посла австрии для выяснения претензий вены  (score: 0.3091355264186859)
2. глава республики алтай олег хорохордин ушел в отставку⁠  (score: 0.21613964438438416)
3. на украине заявили об «очень-очень сильных» повреждениях днепрогэс  (score: 0.2125137597322464)
4. военные восприняли слова макрона как разрешение ударов по регионам россии  (score: 0.20384855568408966)
5. минобороны сообщило о попытке атаки белгородской области ракетой «нептун»  (score: 0.18214498460292816)


In [ ]:
recommendations = get_recommendations(t4)

print(f"Рекомендации по запросу '{t4}':")
for i, (headline, score) in enumerate(recommendations, start=1):
    print(f"{i}. {headline} (score: {score})")

Рекомендации по запросу 'В МИД России вызывали посла Австрии для выяснения претензий Вены':
1. в мид россии вызывали посла австрии для выяснения претензий вены  (score: 0.999653160572052)
2. финляндия рассмотрит вопрос открытия сайменского канала в ближайшее время  (score: 0.29020825028419495)
3. власти киева запретили проводить акцию в поддержку лгбт в метро  (score: 0.28546983003616333)
4. захарова напомнила о конституции сша из-за ситуации со скоттом риттером  (score: 0.2360404133796692)
5. почему мирный план байдена по газе может привести к уходу нетаньяху  (score: 0.21110421419143677)


In [ ]:
recommendations = get_recommendations(t5)

print(f"Рекомендации для запроса '{t5}':")
for i, (headline, score) in enumerate(recommendations, start=1):
    print(f"{i}. {headline} (score: {score})")

Рекомендации для запроса 'Гладков предупредил жителей Белгородской области о «сложной опасности»':
1. офицер ввс греции прибыл в киев для обучения всу работе с f-16  (score: 0.2829465866088867)
2. в швейцарии призвали не превращать июньский саммит в антироссийский  (score: 0.2212703824043274)
3. мэр львова раскритиковал яркие витрины на фоне дефицита электричества  (score: 0.21281112730503082)
4. зеленский пообещал сделать все ради прекращения войны в секторе газа  (score: 0.2060079276561737)
5. захарова напомнила о конституции сша из-за ситуации со скоттом риттером  (score: 0.19992831349372864)
